In [ ]:
# Install required packages (run once)
# !pip install transformers torch torchvision
# !pip install pillow matplotlib numpy opencv-python
# !pip install plotly scikit-learn scipy scikit-image
# !pip install rembg


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from transformers import pipeline
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
from io import BytesIO
import warnings
from sklearn.cluster import KMeans
from scipy.ndimage import binary_dilation, binary_erosion, gaussian_filter
from scipy.spatial.distance import cdist
from skimage import measure, morphology
warnings.filterwarnings('ignore')
import subprocess
import sys


In [ ]:
def install_package(package):
    """Helper function to install packages"""
    try:
        __import__(package)
        print(f"✅ {package} already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} installed successfully")

# Install trimesh and its dependencies
install_package('trimesh')

In [ ]:
try:
    from rembg import remove as rembg_remove
    REMBG_AVAILABLE = True
except ImportError:
    print("⚠️ rembg not available, will use alternative background removal")
    REMBG_AVAILABLE = False

class VoxelBasedImage3DConverter:
    def __init__(self, voxel_resolution=64):
        """
        Initialize the voxel-based 2D to 3D converter

        Args:
            voxel_resolution: Resolution of the voxel grid (default: 64x64x64)
        """
        print("Loading pre-trained models...")
        self.voxel_resolution = voxel_resolution
        self.last_depth_map = None

        # Load MiDaS depth estimation model
        self.depth_estimator = pipeline(
            "depth-estimation",
            model="Intel/dpt-large",
            device=0 if torch.cuda.is_available() else -1
        )
        print("Models loaded successfully!")

    def load_image(self, image_path_or_url):
        """Load image from file path or URL"""
        if image_path_or_url.startswith('http'):
            response = requests.get(image_path_or_url)
            image = Image.open(BytesIO(response.content))
        else:
            image = Image.open(image_path_or_url)
        return image

    def remove_background(self, image):
        """Remove background from image using AI-powered segmentation"""
        print("Removing background...")

        if image.mode != 'RGB':
            rgb_image = image.convert('RGB')
        else:
            rgb_image = image

        if REMBG_AVAILABLE:
            try:
                img_bytes = BytesIO()
                rgb_image.save(img_bytes, format='PNG')
                img_bytes = img_bytes.getvalue()
                output_bytes = rembg_remove(img_bytes)
                segmented_image = Image.open(BytesIO(output_bytes)).convert('RGBA')
                return segmented_image
            except Exception as e:
                print(f"⚠️ rembg failed: {e}")
                return self.remove_background_alternative(rgb_image)
        else:
            return self.remove_background_alternative(rgb_image)

    def remove_background_alternative(self, image):
        """Alternative background removal using clustering"""
        print("Using alternative background removal method...")
        img_array = np.array(image)
        pixels = img_array.reshape(-1, 3)

        kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
        kmeans.fit(pixels)

        labels = kmeans.labels_
        unique_labels, counts = np.unique(labels, return_counts=True)
        bg_label = unique_labels[np.argmax(counts)]

        mask = (labels != bg_label).reshape(img_array.shape[:2])

        rgba_array = np.zeros((img_array.shape[0], img_array.shape[1], 4), dtype=np.uint8)
        rgba_array[:, :, :3] = img_array
        rgba_array[:, :, 3] = mask * 255

        return Image.fromarray(rgba_array, 'RGBA')

    def estimate_depth(self, image):
        """Estimate depth map from 2D image"""
        print("Estimating depth map...")

        if image.mode == 'RGBA':
            rgb_image = Image.new('RGB', image.size, (255, 255, 255))
            rgb_image.paste(image, mask=image.split()[-1])
            depth_input = rgb_image
        else:
            depth_input = image

        depth_result = self.depth_estimator(depth_input)
        depth_map = np.array(depth_result["depth"])
        depth_map = (depth_map - depth_map.min()) / (depth_map.max() - depth_map.min())

        return depth_map

    def detect_spherical_object(self, mask, depth_map):
        """
        Detect if object is likely spherical based on circular mask and depth pattern
        """
        print("Analyzing object shape...")

        # Find contours
        contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if len(contours) == 0:
            return False

        # Get the largest contour
        contour = max(contours, key=cv2.contourArea)

        # Calculate circularity
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)

        if perimeter > 0:
            circularity = 4 * np.pi * area / (perimeter ** 2)
        else:
            return False

        # Get bounding box and check aspect ratio
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = min(w, h) / max(w, h) if max(w, h) > 0 else 0

        # Check depth pattern - spherical objects have higher depth in center
        masked_depth = depth_map * mask
        h_img, w_img = masked_depth.shape

        if len(contour) >= 5:
          ellipse = cv2.fitEllipse(contour)
          (ex, ey), (ma_axis, mi_axis), angle = ellipse
          ellipse_aspect = min(ma_axis, mi_axis) / max(ma_axis, mi_axis) if max(ma_axis, mi_axis) > 0 else 0
        else:
          ellipse_aspect = 0

        masked_depth = depth_map * mask
        h_img, w_img = masked_depth.shape

        # Get center region depth
        center_y, center_x = y + h//2, x + w//2
        radius = min(w, h) // 3
        # center_region = masked_depth[
        #     max(0, center_y-radius):min(h_img, center_y+radius),
        #     max(0, center_x-radius):min(w_img, center_x+radius)
        # ]

        depth_samples = []
        for r_factor in [0, 0.3, 0.6, 0.9]:
          r = int(radius*r_factor)
          if r == 0:
            sample = masked_depth[center_y, center_x] if masked_depth[center_y, center_x] > 0 else 0
          else:
            angles = np.linspace(0, 2*np.pi, 8, endpoint = False)
            samples = []
            for angle in angles:
              sx = int(center_x + r * np.cos(angle))
              sy = int(center_y + r * np.sin(angle))
              if 0 <= sx < w_img and 0 <= sy < h_img and masked_depth[sy, sx] > 0:
                samples.append(masked_depth[sy, sx])
            sample = np.mean(samples) if samples else 0
          depth_samples.append(sample)

        depth_gradient_valid = all(depth_samples[i] >= depth_samples[i+1] * 0.8
                                   for i in range(len(depth_samples)- 1)
                                   if depth_samples[i] > 0 and depth_samples[i+1] > 0)

        # Decision criteria for spherical objects
        is_spherical = (
            circularity > 0.75 and  # Circular shape
            aspect_ratio > 0.85 and  # Nearly square bounding box (more strict for spheres)
            ellipse_aspect > 0.85 and  # Center is significantly deeper than average
            (depth_gradient_valid or depth_samples[0] > np.mean(depth_samples[1:]) * 1.1)
        )

        if circularity > 0.9 and aspect_ratio > 0.9:
          is_spherical = True

        if is_spherical:
            print("🏀 Detected spherical object!")
        else:
            print("📦 Detected non-spherical object")

        return is_spherical

    def estimate_object_depth_scale(self, image, mask=None):
        """
        Estimate appropriate depth scale based on object characteristics.
        Returns both a depth scale factor and a depth mode.
        """
        if image.mode == 'RGBA':
            img_array = np.array(image)[:, :, :3]
        else:
            img_array = np.array(image)

        # Get object dimensions from mask
        if mask is not None:
            # Find bounding box of the object
            rows = np.any(mask, axis=1)
            cols = np.any(mask, axis=0)
            if len(np.where(rows)[0]) > 0 and len(np.where(cols)[0]) > 0:
                y_min, y_max = np.where(rows)[0][[0, -1]]
                x_min, x_max = np.where(cols)[0][[0, -1]]
                obj_height = y_max - y_min
                obj_width = x_max - x_min
                obj_aspect_ratio = obj_width / max(obj_height, 1)

                # Calculate object area as percentage of image
                obj_area = np.sum(mask)
                total_area = mask.shape[0] * mask.shape[1]
                obj_coverage = obj_area / total_area

                # Analyze depth variation in the masked region
                if self.last_depth_map is not None:
                    masked_depth = self.last_depth_map[mask]
                    depth_std = np.std(masked_depth) if masked_depth.size > 0 else 0.5
                else:
                    depth_std = 0.5
            else:
                obj_aspect_ratio = 1.0
                obj_coverage = 0.5
                depth_std = 0.5
        else:
            obj_aspect_ratio = img_array.shape[1] / img_array.shape[0]
            obj_coverage = 0.5
            depth_std = 0.5

        # Determine object type and appropriate depth scale
        depth_mode = "normal"
        if obj_aspect_ratio > 3.0:  # Very wide and flat objects (guns from side, swords)
            depth_scale = 0.08
            depth_mode = "ultra_thin"
        elif obj_aspect_ratio > 2.2:  # Wide flat objects (laptops, phones)
            depth_scale = 0.12
            depth_mode = "very_thin"
        elif obj_aspect_ratio > 1.6:  # Moderately wide (cars from side)
            depth_scale = 0.18
            depth_mode = "thin"
        elif obj_aspect_ratio > 1.2:  # Slightly wide
            depth_scale = 0.25
            depth_mode = "normal"
        else:  # Square or tall objects (people, bottles)
            depth_scale = 0.35
            depth_mode = "full"

        # Adjust based on coverage
        if obj_coverage > 0.5:  # Large object - might need more depth
            depth_scale *= 1.1
        elif obj_coverage < 0.15:  # Small object - reduce depth
            depth_scale *= 0.7

        # Adjust based on depth variance
        if depth_std < 0.2:
            depth_scale *= 0.6  # Flat object detected

        # Clamp to reasonable range
        depth_scale = np.clip(depth_scale, 0.05, 0.5)

        print(f"  Object analysis: aspect_ratio={obj_aspect_ratio:.2f}, coverage={obj_coverage:.2f}")
        print(f"  Depth mode: {depth_mode}, depth_scale={depth_scale:.3f}")

        return depth_scale, depth_mode

    def create_spherical_voxel_grid(self, image, mask, preserve_width=True):

      """
      Create voxel grid specifically for spherical objects like basketballs
      with TRUE spherical geometry
      """
      print("Creating spherical voxel grid...")

    # Find object bounds from mask
      contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
      if len(contours) == 0:
          return np.zeros((self.voxel_resolution, self.voxel_resolution, self.voxel_resolution, 4))

    # Get the largest contour
      contour = max(contours, key=cv2.contourArea)

    # Get minimum enclosing circle
      (center_x, center_y), radius = cv2.minEnclosingCircle(contour)

    # For a true sphere, all dimensions should be equal based on the diameter
    # The voxel grid should be cubic for a sphere
      sphere_size = int(self.voxel_resolution)

    # Create cubic voxel grid for sphere
      voxel_height = sphere_size
      voxel_width = sphere_size
      voxel_depth = sphere_size

      print(f"  True sphere voxel dimensions: {voxel_height}x{voxel_width}x{voxel_depth}")

      voxel_grid = np.zeros((voxel_height, voxel_width, voxel_depth, 4))

    # Calculate the radius in voxel space (should be consistent across all dimensions)
      voxel_radius = sphere_size * 0.45  # Slightly less than half to fit nicely

    # Center of the voxel grid
      center_voxel_x = voxel_width / 2.0
      center_voxel_y = voxel_height / 2.0
      center_voxel_z = voxel_depth / 2.0

    # Prepare image for texture mapping
      if image.mode == 'RGBA':
          img_array = np.array(image)
      else:
          img_array = np.array(image.convert('RGBA'))

    # Get the bounding box of the mask for better texture mapping
      rows = np.any(mask, axis=1)
      cols = np.any(mask, axis=0)
      rmin, rmax = np.where(rows)[0][[0, -1]]
      cmin, cmax = np.where(cols)[0][[0, -1]]

    # Crop and resize the texture to focus on the object
      cropped_img = img_array[rmin:rmax+1, cmin:cmax+1]
      texture_size = max(cropped_img.shape[0], cropped_img.shape[1])

    # Make texture square by padding if needed
      if cropped_img.shape[0] < texture_size or cropped_img.shape[1] < texture_size:
          pad_y = (texture_size - cropped_img.shape[0]) // 2
          pad_x = (texture_size - cropped_img.shape[1]) // 2
          cropped_img = np.pad(cropped_img,
                            ((pad_y, texture_size - cropped_img.shape[0] - pad_y),
                             (pad_x, texture_size - cropped_img.shape[1] - pad_x),
                             (0, 0)),
                            mode='constant')

    # Resize texture to voxel dimensions for easier mapping
      img_resized = cv2.resize(cropped_img, (sphere_size, sphere_size))

      print(f"  Creating true sphere: center=({center_voxel_x:.1f}, {center_voxel_y:.1f}, {center_voxel_z:.1f}), radius={voxel_radius:.1f}")

    # Create the sphere with proper 3D structure
      for y in range(voxel_height):
          for x in range(voxel_width):
              for z in range(voxel_depth):
                # Calculate distance from center in 3D space
                  dist = np.sqrt((x - center_voxel_x)**2 +
                             (y - center_voxel_y)**2 +
                             (z - center_voxel_z)**2)

                # Check if inside sphere
                  if dist <= voxel_radius:
                    # Calculate spherical coordinates for texture mapping
                      if dist > 0:
                        # Get the normalized vector from center
                          dx = (x - center_voxel_x) / voxel_radius
                          dy = (y - center_voxel_y) / voxel_radius
                          dz = (z - center_voxel_z) / voxel_radius

                        # Convert to spherical coordinates for texture mapping
                        # Use equirectangular projection
                          theta = np.arctan2(dy, dx)  # Azimuthal angle
                          phi = np.arccos(np.clip(dz / (dist/voxel_radius), -1, 1))  # Polar angle

                        # Map to texture coordinates
                          tex_x = int((theta / (2 * np.pi) + 0.5) * (sphere_size - 1))
                          tex_y = int((phi / np.pi) * (sphere_size - 1))

                        # Alternative simpler mapping for front-facing texture
                        # Project onto front face for main texture
                          if z >= center_voxel_z - voxel_radius * 0.3:  # Front hemisphere gets main texture
                            # Use orthographic projection for front
                              tex_x_alt = int(x)
                              tex_y_alt = int(y)
                              if 0 <= tex_x_alt < sphere_size and 0 <= tex_y_alt < sphere_size:
                                  tex_x = tex_x_alt
                                  tex_y = tex_y_alt
                      else:
                          tex_x = int(center_voxel_x)
                          tex_y = int(center_voxel_y)

                    # Clamp texture coordinates
                      tex_x = max(0, min(sphere_size - 1, tex_x))
                      tex_y = max(0, min(sphere_size - 1, tex_y))

                    # Get color from texture
                      color = img_resized[tex_y, tex_x][:3] / 255.0

                    # Add subtle shading based on position to enhance 3D effect
                    # Light coming from top-front-right
                      light_dir = np.array([1, -1, 1])
                      light_dir = light_dir / np.linalg.norm(light_dir)

                      if dist > 0:
                          normal = np.array([x - center_voxel_x,
                                         y - center_voxel_y,
                                         z - center_voxel_z]) / dist
                          diffuse = max(0, np.dot(normal, light_dir))
                          ambient = 0.6
                          lighting = ambient + (1 - ambient) * diffuse
                      else:
                          lighting = 1.0

                    # Apply color with lighting
                      voxel_grid[y, x, z, :3] = color * lighting
                      voxel_grid[y, x, z, 3] = 1.0

    # Optional: Add some surface roughness for basketball texture
    # This creates the characteristic bumpy surface
      if "basketball" in str(image.filename).lower() if hasattr(image, 'filename') else False:
          occupied = voxel_grid[:, :, :, 3] > 0
          for y in range(voxel_height):
              for x in range(voxel_width):
                  for z in range(voxel_depth):
                      if occupied[y, x, z]:
                          dist = np.sqrt((x - center_voxel_x)**2 +
                                     (y - center_voxel_y)**2 +
                                     (z - center_voxel_z)**2)
                        # Add subtle noise to surface voxels
                          if dist > voxel_radius * 0.9:
                            noise = np.sin(x * 0.5) * np.sin(y * 0.5) * 0.1
                            voxel_grid[y, x, z, :3] *= (1 + noise)

    # Store dimensions
      self.voxel_dims = (voxel_height, voxel_width, voxel_depth)

      print(f"  Sphere created with {(voxel_grid[:, :, :, 3] > 0).sum()} voxels")
      return voxel_grid



    def create_voxel_grid(self, image, depth_map, mask=None, preserve_width=True, auto_depth=True):
        """
        Create a voxel grid for non-spherical objects with intelligent depth estimation
        """
        print(f"Creating voxel grid ({self.voxel_resolution}³)...")

        # Store depth map for analysis
        self.last_depth_map = depth_map

        # Get original image dimensions
        original_h, original_w = depth_map.shape
        aspect_ratio = original_w / original_h

        # Estimate appropriate depth scale for this object
        if auto_depth:
            depth_scale, depth_mode = self.estimate_object_depth_scale(image, mask)
        else:
            depth_scale = 0.3
            depth_mode = "manual"

        # Determine voxel grid dimensions based on aspect ratio
        if preserve_width:
            voxel_height = self.voxel_resolution
            voxel_width = int(self.voxel_resolution * aspect_ratio)

            # Adjust voxel depth based on detected object type
            if depth_mode == "ultra_thin":
                voxel_depth = max(8, int(self.voxel_resolution * 0.15))
            elif depth_mode == "very_thin":
                voxel_depth = max(12, int(self.voxel_resolution * 0.25))
            elif depth_mode == "thin":
                voxel_depth = max(16, int(self.voxel_resolution * 0.35))
            elif depth_mode == "normal":
                voxel_depth = max(24, int(self.voxel_resolution * 0.5))
            else:  # "full"
                voxel_depth = int(self.voxel_resolution * 0.7)

            # Ensure width doesn't exceed reasonable bounds
            if voxel_width > self.voxel_resolution * 2:
                voxel_width = self.voxel_resolution * 2
                voxel_height = int(voxel_width / aspect_ratio)
        else:
            voxel_height = self.voxel_resolution
            voxel_width = self.voxel_resolution
            voxel_depth = int(self.voxel_resolution * depth_scale * 2)

        print(f"  Voxel dimensions: {voxel_height}x{voxel_width}x{voxel_depth} (depth mode: {depth_mode})")

        # Initialize voxel grid with appropriate dimensions
        voxel_grid = np.zeros((voxel_height, voxel_width, voxel_depth, 4))

        # Resize inputs to match voxel dimensions
        depth_resized = cv2.resize(depth_map, (voxel_width, voxel_height))

        if image.mode == 'RGBA':
            img_array = np.array(image)
        else:
            img_array = np.array(image.convert('RGBA'))

        img_resized = cv2.resize(img_array, (voxel_width, voxel_height))

        if mask is not None:
            mask_resized = cv2.resize(mask.astype(np.uint8),
                                     (voxel_width, voxel_height)).astype(bool)
        else:
            mask_resized = np.ones((voxel_height, voxel_width), dtype=bool)

        # Create depth-based voxels with conservative depth mapping
        for y in range(voxel_height):
            for x in range(voxel_width):
                if mask_resized[y, x]:
                    # Get depth value
                    depth_value = depth_resized[y, x]

                    # Center point in Z dimension
                    z_center = voxel_depth // 2

                    # Calculate depth extent based on mode
                    if depth_mode == "ultra_thin":
                        depth_extent = max(1, int(depth_value * voxel_depth * 0.15))
                    elif depth_mode == "very_thin":
                        depth_extent = max(2, int(depth_value * voxel_depth * 0.2))
                    elif depth_mode == "thin":
                        depth_extent = max(3, int(depth_value * voxel_depth * 0.3))
                    else:
                        depth_extent = max(4, int(depth_value * voxel_depth * 0.4))

                    # Ensure we don't exceed bounds
                    depth_extent = min(depth_extent, voxel_depth // 2 - 1)

                    # Create symmetrical depth profile
                    z_front = min(voxel_depth - 1, z_center + depth_extent)
                    z_back = max(0, z_center - depth_extent)

                    # Get color for this position
                    color = img_resized[y, x][:3] / 255.0

                    # Fill the voxels with color
                    for z in range(z_back, z_front + 1):
                        # Calculate falloff
                        if depth_extent > 0:
                            distance_from_center = abs(z - z_center) / depth_extent
                            if depth_mode in ["ultra_thin", "very_thin"]:
                                alpha = 1.0 - (distance_from_center * 0.4)
                            else:
                                alpha = 1.0 - (distance_from_center * 0.2)
                        else:
                            alpha = 1.0

                        voxel_grid[y, x, z, :3] = color
                        voxel_grid[y, x, z, 3] = alpha

        # Apply morphological operations
        occupied = voxel_grid[:, :, :, 3] > 0.5

        # Fill internal voxels
        for y in range(voxel_height):
            for x in range(voxel_width):
                occupied_z = np.where(occupied[y, x, :])[0]
                if len(occupied_z) > 1:
                    z_min, z_max = occupied_z[0], occupied_z[-1]
                    for z in range(z_min, z_max + 1):
                        if voxel_grid[y, x, z, 3] < 0.5:
                            voxel_grid[y, x, z, :3] = voxel_grid[y, x, z_max, :3]
                            voxel_grid[y, x, z, 3] = 1.0

        # Apply gentler smoothing for thin objects
        if depth_mode not in ["ultra_thin", "very_thin"]:
            kernel_size = 3
            struct_elem = np.ones((kernel_size, kernel_size, kernel_size))
            occupied = binary_dilation(occupied, structure=struct_elem, iterations=1)
            occupied = binary_erosion(occupied, structure=struct_elem, iterations=1)

            # Update voxel grid with smoothed occupancy
            new_voxel_grid = np.zeros_like(voxel_grid)
            for y in range(voxel_height):
                for x in range(voxel_width):
                    for z in range(voxel_depth):
                        if occupied[y, x, z] and voxel_grid[y, x, z, 3] > 0:
                            new_voxel_grid[y, x, z] = voxel_grid[y, x, z]

            voxel_grid = new_voxel_grid

        # Store dimensions for later use
        self.voxel_dims = (voxel_height, voxel_width, voxel_depth)

        return voxel_grid

    def voxels_to_cubes(self, voxel_grid, cube_size=1.0):
        """
        Convert voxel grid to cube representations (LEGO bricks)
        """
        print("Converting voxels to LEGO-style cubes...")

        voxel_height, voxel_width, voxel_depth = voxel_grid.shape[:3]
        occupied_voxels = np.where(voxel_grid[:, :, :, 3] > 0)

        cube_positions = []
        cube_colors = []

        # Sample voxels for performance
        total_voxels = len(occupied_voxels[0])
        sampling_rate = max(1, total_voxels // 5000)

        for idx in range(0, total_voxels, sampling_rate):
            y = occupied_voxels[0][idx]
            x = occupied_voxels[1][idx]
            z = occupied_voxels[2][idx]

            # Convert to centered coordinates
            pos_x = (x - voxel_width / 2) * cube_size
            pos_y = (voxel_height / 2 - y) * cube_size
            pos_z = (z - voxel_depth / 2) * cube_size

            cube_positions.append([pos_x, pos_y, pos_z])
            cube_colors.append(voxel_grid[y, x, z, :3])

        return np.array(cube_positions), np.array(cube_colors)

    def voxels_to_mesh(self, voxel_grid):
        """
        Convert voxel grid to mesh using marching cubes algorithm
        """
        print("Converting voxels to mesh...")

        voxel_height, voxel_width, voxel_depth = voxel_grid.shape[:3]

        # Extract occupancy grid
        occupancy = voxel_grid[:, :, :, 3]

        try:
            verts, faces, normals, values = measure.marching_cubes(
                occupancy, level=0.5, spacing=(1.0, 1.0, 1.0)
            )

            # Center the mesh
            verts = verts - [voxel_height/2, voxel_width/2, voxel_depth/2]

            # Extract colors for vertices
            colors = []
            for vert in verts:
                voxel_pos = (vert + [voxel_height/2, voxel_width/2, voxel_depth/2]).astype(int)
                voxel_pos[0] = np.clip(voxel_pos[0], 0, voxel_height - 1)
                voxel_pos[1] = np.clip(voxel_pos[1], 0, voxel_width - 1)
                voxel_pos[2] = np.clip(voxel_pos[2], 0, voxel_depth - 1)

                color = voxel_grid[voxel_pos[0], voxel_pos[1], voxel_pos[2], :3]
                colors.append(color)

            return verts, faces, np.array(colors)
        except Exception as e:
            print(f"Warning: Marching cubes failed: {e}")
            return None, None, None

    def visualize_voxel_model(self, original_image, voxel_grid, depth_map,
                             visualization_mode='cubes', object_type='standard'):
        """
        Visualize the voxel-based 3D model
        """
        voxel_height, voxel_width, voxel_depth = voxel_grid.shape[:3]

        fig = make_subplots(
            rows=2, cols=2,
            specs=[[{"type": "image"}, {"type": "image"}],
                   [{"type": "image"}, {"type": "scatter3d"}]],
            subplot_titles=("Original Image", "Depth Map",
                          f"Voxel Slice ({object_type})", f"3D Model ({visualization_mode})"),
            vertical_spacing=0.08
        )

        # Original image
        fig.add_trace(
            go.Image(z=np.array(original_image)),
            row=1, col=1
        )

        # Depth map
        fig.add_trace(
            go.Heatmap(z=depth_map, colorscale='viridis'),
            row=1, col=2
        )

        # Voxel slice (middle Z layer)
        middle_slice = voxel_grid[:, :, voxel_depth//2, 3]
        fig.add_trace(
            go.Heatmap(z=middle_slice, colorscale='gray'),
            row=2, col=1
        )

        # 3D Model
        if visualization_mode == 'cubes':
            cube_positions, cube_colors = self.voxels_to_cubes(voxel_grid)

            if len(cube_positions) > 0:
                fig.add_trace(
                    go.Scatter3d(
                        x=cube_positions[:, 0],
                        y=cube_positions[:, 1],
                        z=cube_positions[:, 2],
                        mode='markers',
                        marker=dict(
                            size=8,
                            color=cube_colors,
                            opacity=0.9,
                            symbol='square'
                        )
                    ),
                    row=2, col=2
                )

        elif visualization_mode == 'mesh':
            verts, faces, colors = self.voxels_to_mesh(voxel_grid)

            if verts is not None and len(faces) > 0:
                fig.add_trace(
                    go.Mesh3d(
                        x=verts[:, 1],  # Swap x and y for correct orientation
                        y=verts[:, 0],
                        z=verts[:, 2],
                        i=faces[:, 0],
                        j=faces[:, 1],
                        k=faces[:, 2],
                        vertexcolor=colors,
                        opacity=0.9,
                        lighting=dict(
                            ambient=0.4,
                            diffuse=0.8,
                            specular=0.5,
                            roughness=0.2
                        ),
                        lightposition=dict(x=100, y=200, z=50)
                    ),
                    row=2, col=2
                )

        # Calculate aspect ratio for 3D scene
        aspect_ratio = voxel_width / voxel_height
        depth_ratio = voxel_depth / voxel_height

        object_emoji = "🏀" if object_type == 'spherical' else "📦"

        fig.update_layout(
            title=f"{object_emoji} Voxel-Based 3D Model ({object_type.capitalize()}) - {voxel_height}x{voxel_width}x{voxel_depth}",
            height=800,
            scene=dict(
                xaxis_title="X",
                yaxis_title="Y",
                zaxis_title="Z",
                aspectmode='manual',
                aspectratio=dict(x=aspect_ratio, y=1, z=depth_ratio),
                camera=dict(
                    eye=dict(x=1.5, y=1.5, z=1.5)
                )
            )
        )

        fig.show()

    def process_image(self, image_path_or_url, remove_background=True,
                     visualization_mode='cubes', voxel_resolution=None,
                     preserve_width=True, auto_depth=True, force_mode=None):
        """
        Complete pipeline to convert 2D image to voxel-based 3D model

        Args:
            image_path_or_url: Path to image or URL
            remove_background: Whether to remove background
            visualization_mode: 'cubes' for LEGO-style, 'mesh' for smooth
            voxel_resolution: Override default voxel resolution
            preserve_width: If True, maintains original image aspect ratio in 3D
            auto_depth: If True, automatically adjusts depth based on object type
            force_mode: Force 'spherical' or 'standard' mode (None for auto-detect)
        """
        try:
            if voxel_resolution:
                self.voxel_resolution = voxel_resolution

            # Load image
            print(f"Loading image from: {image_path_or_url}")
            original_image = self.load_image(image_path_or_url)

            print(f"Original image size: {original_image.size[0]}x{original_image.size[1]}")

            # Remove background if requested
            if remove_background:
                if original_image.mode == 'RGBA':
                    alpha = np.array(original_image)[:, :, 3]
                    if np.any(alpha < 255):
                        print("✅ Image already has transparent background")
                        working_image = original_image
                    else:
                        working_image = self.remove_background(original_image)
                else:
                    working_image = self.remove_background(original_image)

                # Create mask from alpha channel
                alpha = np.array(working_image)[:, :, 3]
                mask = alpha > 50
            else:
                working_image = original_image
                mask = None

            # Estimate depth
            depth_map = self.estimate_depth(working_image)

            # Apply mask to depth if available
            if mask is not None:
                depth_map = depth_map * mask

            # Detect object type or use forced mode
            if force_mode == 'spherical':
                is_spherical = True
                print("🔧 Forced spherical mode")
            elif force_mode == 'standard':
                is_spherical = False
                print("🔧 Forced standard mode")
            else:
                # Auto-detect
                is_spherical = self.detect_spherical_object(mask, depth_map) if mask is not None else False

            # Create appropriate voxel grid
            if is_spherical:
                print("🏀 Using spherical reconstruction...")
                voxel_grid = self.create_spherical_voxel_grid(working_image, mask, preserve_width)
                object_type = 'spherical'
            else:
                print("📦 Using standard reconstruction with intelligent depth...")
                voxel_grid = self.create_voxel_grid(working_image, depth_map, mask,
                                                   preserve_width=preserve_width,
                                                   auto_depth=auto_depth)
                object_type = 'standard'

            # Visualize results
            self.visualize_voxel_model(original_image, voxel_grid, depth_map,
                                      visualization_mode, object_type)

            print(f"✅ Voxel-based 3D model generation completed!")
            print(f"📊 Voxel dimensions: {self.voxel_dims[0]}x{self.voxel_dims[1]}x{self.voxel_dims[2]}")
            print(f"🎨 Visualization mode: {visualization_mode}")
            print(f"🎯 Object type: {object_type}")
            print(f"📐 Width preservation: {'Enabled' if preserve_width else 'Disabled'}")
            print(f"🎯 Auto depth: {'Enabled' if auto_depth else 'Disabled'}")

            return {
                'original_image': original_image,
                'depth_map': depth_map,
                'voxel_grid': voxel_grid,
                'mask': mask,
                'object_type': object_type,
                'success': True
            }

        except Exception as e:
            print(f"❌ Error processing image: {str(e)}")
            import traceback
            traceback.print_exc()
            return {'success': False, 'error': str(e)}

# Initialize the converter
print("🚀 Initializing Advanced Voxel-Based 3D Converter...")
converter = VoxelBasedImage3DConverter(voxel_resolution=64)

# Helper functions for easy use
def upload_and_process_voxel(remove_bg=True, visualization_mode='cubes',
                             voxel_resolution=64, preserve_width=True,
                             auto_depth=True, force_mode=None):
    """
    Upload and process image to create voxel-based 3D model

    Args:
        remove_bg: Remove background (default: True)
        visualization_mode: 'cubes' for LEGO-style, 'mesh' for smooth
        voxel_resolution: Resolution of voxel grid (higher = more detail)
        preserve_width: If True, maintains original image width in 3D model
        auto_depth: If True, automatically adjusts depth based on object type
        force_mode: Force 'spherical' or 'standard' mode (None for auto-detect)
    """
    from google.colab import files
    print("📁 Click 'Choose Files' to upload your image...")
    uploaded = files.upload()

    if uploaded:
        filename = list(uploaded.keys())[0]
        print(f"\n🔄 Processing uploaded image: {filename}")

        converter.voxel_resolution = voxel_resolution

        result = converter.process_image(
            filename,
            remove_background=remove_bg,
            visualization_mode=visualization_mode,
            preserve_width=preserve_width,
            auto_depth=auto_depth,
            force_mode=force_mode
        )
        return result
    else:
        print("❌ No file uploaded!")
        return None

def process_voxel_image(image_path, remove_bg=True, visualization_mode='cubes',
                       voxel_resolution=64, preserve_width=True,
                       auto_depth=True, force_mode=None):
    """
    Process image to create voxel-based 3D model

    Args:
        image_path: Path to image or URL
        remove_bg: Remove background (default: True)
        visualization_mode: 'cubes' for LEGO-style, 'mesh' for smooth
        voxel_resolution: Resolution of voxel grid
        preserve_width: If True, maintains original image width in 3D model
        auto_depth: If True, automatically adjusts depth based on object type
        force_mode: Force 'spherical' or 'standard' mode (None for auto-detect)
    """
    print(f"\n🔄 Processing image: {image_path}")

    converter.voxel_resolution = voxel_resolution

    result = converter.process_image(
        image_path,
        remove_background=remove_bg,
        visualization_mode=visualization_mode,
        preserve_width=preserve_width,
        auto_depth=auto_depth,
        force_mode=force_mode
    )
    return result

# Usage examples
print("\n🎯 ADVANCED 3D MODEL GENERATOR READY!")
print("=" * 60)
print("🆕 COMBINED FEATURES:")
print("• AUTO-DETECTS spherical objects (basketballs, balls)")
print("• INTELLIGENT DEPTH for non-spherical objects (guns, phones, cars)")
print("• WIDTH PRESERVATION maintains original proportions")
print("• Supports both LEGO-style and smooth mesh visualization")
print("=" * 60)
print("\n📌 Quick Start Commands:")
print("\n1️⃣ Auto-detect with intelligent depth (recommended):")
print("result = upload_and_process_voxel(")
print("    remove_bg=True,")
print("    visualization_mode='mesh',")
print("    voxel_resolution=64,")
print("    preserve_width=True,  # Maintains proportions")
print("    auto_depth=True       # Smart depth adjustment")
print(")")
print("\n2️⃣ Force spherical mode (for balls):")
print("result = upload_and_process_voxel(")
print("    remove_bg=True,")
print("    visualization_mode='mesh',")
print("    force_mode='spherical'")
print(")")
print("\n3️⃣ Force standard mode (for non-spherical):")
print("result = upload_and_process_voxel(")
print("    remove_bg=True,")
print("    visualization_mode='cubes',")
print("    force_mode='standard',")
print("    auto_depth=True  # Will detect if gun/thin object")
print(")")
print("\n🎯 Object Type Detection:")
print("• SPHERICAL: Basketballs, soccer balls → True sphere")
print("• Ultra-thin (AR > 3.0): Guns, swords → depth ~8-15%")
print("• Very thin (AR > 2.2): Phones, cards → depth ~12-25%")
print("• Thin (AR > 1.6): Cars from side → depth ~18-35%")
print("• Normal (AR > 1.2): Regular objects → depth ~25-50%")
print("• Full (AR ≤ 1.2): People, bottles → depth ~35-70%")
print("\n✅ The system will automatically:")
print("• Create perfect spheres for balls")
print("• Use intelligent depth for guns and thin objects")
print("• Preserve original proportions")
print("\n✅ Setup complete! Upload your image!")

In [ ]:
import trimesh
import os
from datetime import datetime
import numpy as np
from skimage import measure
from PIL import Image

def create_enhanced_mtl_file(obj_path, mesh):
    """Create an enhanced MTL file with texture mapping for Blender"""
    mtl_path = obj_path.replace('.obj', '.mtl')
    texture_path = obj_path.replace('.obj', '_texture.png')

    # Extract vertex colors and create a texture image
    if hasattr(mesh, 'vertex_colors') and mesh.vertex_colors is not None:
        # Create texture from vertex colors
        create_texture_from_vertex_colors(mesh, texture_path)

        # Write enhanced MTL file
        with open(mtl_path, 'w') as f:
            f.write("# Enhanced MTL file for Blender compatibility\n")
            f.write("# Generated by VoxelBasedImage3DConverter\n\n")

            f.write("newmtl material0\n")
            f.write("Ka 1.0 1.0 1.0\n")  # Ambient color
            f.write("Kd 1.0 1.0 1.0\n")  # Diffuse color
            f.write("Ks 0.1 0.1 0.1\n")  # Specular color
            f.write("Ns 10.0\n")          # Specular exponent
            f.write("d 1.0\n")            # Transparency
            f.write("illum 2\n")          # Illumination model

            # Add texture map
            texture_filename = os.path.basename(texture_path)
            f.write(f"map_Kd {texture_filename}\n")  # Diffuse texture map

        print(f"  Enhanced MTL file created: {mtl_path}")
        print(f"  Texture file created: {texture_path}")
    else:
        print("  No vertex colors found, creating basic MTL")

def create_texture_from_vertex_colors(mesh, texture_path):
    """Create a texture image from vertex colors"""
    if not hasattr(mesh, 'vertex_colors') or mesh.vertex_colors is None:
        return

    # Get vertex colors
    colors = mesh.vertex_colors
    if colors.max() <= 1.0:
        colors = (colors * 255).astype(np.uint8)

    # Create a simple texture by mapping vertex colors
    # For a basketball, we'll create a spherical texture map
    texture_size = 512

    # Create UV sphere mapping
    vertices = mesh.vertices

    # Normalize vertices to unit sphere
    vertex_norms = np.linalg.norm(vertices, axis=1)
    normalized_verts = vertices / vertex_norms[:, np.newaxis]

    # Convert to spherical coordinates for UV mapping
    u_coords = 0.5 + np.arctan2(normalized_verts[:, 0], normalized_verts[:, 2]) / (2 * np.pi)
    v_coords = 0.5 - np.arcsin(normalized_verts[:, 1]) / np.pi

    # Create texture image
    texture_img = np.zeros((texture_size, texture_size, 3), dtype=np.uint8)

    # Map vertex colors to texture coordinates
    for i, (u, v, color) in enumerate(zip(u_coords, v_coords, colors)):
        tex_x = int(np.clip(u * (texture_size - 1), 0, texture_size - 1))
        tex_y = int(np.clip(v * (texture_size - 1), 0, texture_size - 1))

        # Blend colors if multiple vertices map to same pixel
        if texture_img[tex_y, tex_x].sum() == 0:
            texture_img[tex_y, tex_x] = color[:3]
        else:
            texture_img[tex_y, tex_x] = (texture_img[tex_y, tex_x] + color[:3]) // 2

    # Fill empty pixels with nearest neighbor
    mask = texture_img.sum(axis=2) == 0
    if mask.any():
        from scipy.ndimage import distance_transform_edt

        # For each channel
        for channel in range(3):
            channel_data = texture_img[:, :, channel]
            invalid_mask = channel_data == 0

            if invalid_mask.any():
                # Use distance transform to fill gaps
                indices = distance_transform_edt(invalid_mask, return_distances=False, return_indices=True)
                channel_data[invalid_mask] = channel_data[tuple(indices[:, invalid_mask])]
                texture_img[:, :, channel] = channel_data

    # Save texture image
    img = Image.fromarray(texture_img)
    img.save(texture_path)

def export_voxel_mesh(self, voxel_grid, filename=None, format='obj', simplify=True, smooth=True):
    """
    Export the voxel grid as a 3D mesh file (OBJ, STL, PLY, or GLTF)

    Args:
        voxel_grid: The voxel grid to export
        filename: Output filename (without extension)
        format: File format ('obj', 'stl', 'ply', 'gltf', 'glb')
        simplify: Whether to simplify the mesh (reduce polygons)
        smooth: Whether to smooth the mesh

    Returns:
        Path to the exported file
    """
    print(f"Exporting mesh to {format.upper()} format...")

    # Extract mesh from voxels using marching cubes
    voxel_height, voxel_width, voxel_depth = voxel_grid.shape[:3]
    occupancy = voxel_grid[:, :, :, 3]

    try:
        # Generate mesh using marching cubes
        verts, faces, normals, values = measure.marching_cubes(
            occupancy, level=0.5, spacing=(1.0, 1.0, 1.0)
        )

        # Center the mesh
        verts = verts - [voxel_height/2, voxel_width/2, voxel_depth/2]

        # Scale to reasonable size (important for Blender)
        scale_factor = 0.1  # Adjust this to change size in Blender
        verts = verts * scale_factor

        # Extract vertex colors
        vertex_colors = []
        for vert_original in (verts / scale_factor + [voxel_height/2, voxel_width/2, voxel_depth/2]):
            voxel_pos = vert_original.astype(int)
            voxel_pos[0] = np.clip(voxel_pos[0], 0, voxel_height - 1)
            voxel_pos[1] = np.clip(voxel_pos[1], 0, voxel_width - 1)
            voxel_pos[2] = np.clip(voxel_pos[2], 0, voxel_depth - 1)

            color = voxel_grid[voxel_pos[0], voxel_pos[1], voxel_pos[2], :3]
            # Convert to 0-255 range for standard formats
            vertex_colors.append((color * 255).astype(np.uint8))

        vertex_colors = np.array(vertex_colors)

        # Create trimesh object
        mesh = trimesh.Trimesh(
            vertices=verts,
            faces=faces,
            vertex_colors=vertex_colors,
            process=False  # Don't process immediately to preserve colors
        )

        # Apply smoothing if requested
        if smooth:
            print("  Applying Laplacian smoothing...")
            try:
                # FIXED: Import smoothing module and assign only the vertices
                import trimesh.smoothing as smoothing
                # Get the smoothed vertices (returns numpy array, not mesh)
                smoothed_vertices = smoothing.filter_laplacian(mesh, lamb=0.5, iterations=10)
                # Assign only the vertices
                mesh.vertices = smoothed_vertices

            except Exception as smooth_error:
                print(f"  Warning: Smoothing failed ({smooth_error}), continuing without smoothing...")
                # Alternative simple smoothing using built-in methods
                try:
                    mesh = mesh.smoothed()
                except:
                    print("  Fallback smoothing also failed, using original mesh")

        # Simplify mesh if requested (reduce polygon count)
        if simplify and len(mesh.faces) > 10000:
            print(f"  Simplifying mesh (current faces: {len(mesh.faces)})...")
            target_faces = min(10000, len(mesh.faces) // 2)
            try:
                mesh = mesh.simplify_quadric_decimation(target_faces)
                print(f"  Simplified to {len(mesh.faces)} faces")
            except Exception as simplify_error:
                print(f"  Warning: Simplification failed ({simplify_error}), using original mesh")

        # Fix mesh normals
        mesh.fix_normals()

        # Generate filename if not provided
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"voxel_model_{timestamp}"

        # Ensure output directory exists
        output_dir = "3d_exports"
        os.makedirs(output_dir, exist_ok=True)

        # Full path with extension
        output_path = os.path.join(output_dir, f"{filename}.{format}")

        # Export based on format
        if format == 'obj':
            # OBJ format - create better material support for Blender
            mesh.export(output_path)

            # Create enhanced MTL file for better Blender compatibility
            create_enhanced_mtl_file(output_path, mesh)

            print(f"  Exported OBJ with enhanced materials: {output_path}")

        elif format == 'stl':
            # STL format - good for 3D printing
            mesh.export(output_path)
            print(f"  Exported STL: {output_path}")

        elif format == 'ply':
            # PLY format - includes vertex colors, better Blender support
            # Ensure vertex colors are properly formatted for Blender
            if hasattr(mesh, 'vertex_colors') and mesh.vertex_colors is not None:
                # Ensure colors are in 0-255 range
                colors = mesh.vertex_colors
                if colors.max() <= 1.0:
                    mesh.vertex_colors = (colors * 255).astype(np.uint8)

            mesh.export(output_path)
            print(f"  Exported PLY with vertex colors: {output_path}")
            print("  Note: PLY format preserves vertex colors - manual material setup required in Blender")

        elif format == 'glb':
            # GLB format - binary GLTF with embedded textures
            mesh.export(output_path)
            print(f"  Exported GLB with embedded materials: {output_path}")
            print("  Note: GLB format embeds all textures and has excellent Blender support")

        else:
            raise ValueError(f"Unsupported format: {format}")

        print(f"Mesh exported successfully!")
        print(f"File saved to: {output_path}")
        print(f"Mesh statistics:")
        print(f"   - Vertices: {len(mesh.vertices)}")
        print(f"   - Faces: {len(mesh.faces)}")
        print(f"   - Bounding box: {mesh.bounds}")

        return output_path

    except Exception as e:
        print(f"Error exporting mesh: {str(e)}")
        import traceback
        traceback.print_exc()
        return None


def download_mesh(self, voxel_grid, format='obj'):
    """
    Export and prepare mesh for download in Google Colab

    Args:
        voxel_grid: The voxel grid to export
        format: Export format ('obj', 'stl', 'ply', 'gltf', 'glb')
    """
    # Export the mesh
    output_path = self.export_voxel_mesh(voxel_grid, format=format)

    if output_path and os.path.exists(output_path):
        # In Google Colab, trigger download
        try:
            from google.colab import files
            files.download(output_path)

            # If OBJ, also download the MTL file if it exists
            if format == 'obj':
                mtl_path = output_path.replace('.obj', '.mtl')
                if os.path.exists(mtl_path):
                    files.download(mtl_path)

            print(f"\nFile ready for download!")

        except ImportError:
            print(f"\nFile exported to: {output_path}")
            print("(Download not available outside Google Colab)")

    return output_path


def download_mesh_for_blender(self, voxel_grid, format='ply', filename=None):
    """
    Download mesh in the best format for Blender with textures

    Args:
        voxel_grid: The voxel grid to export
        format: 'ply' (recommended), 'obj', or 'stl'
        filename: Optional custom filename
    """
    print("Exporting for Blender with texture support...")

    recommended_formats = ['ply', 'obj']
    if format not in ['obj', 'stl', 'ply']:
        print(f"Warning: Format '{format}' not supported. Using 'ply' instead.")
        format = 'ply'

    if format not in recommended_formats:
        print(f"Note: For best Blender texture support, consider using 'ply' or 'obj' format")

    # Export the mesh
    output_path = self.export_voxel_mesh(voxel_grid, filename=filename, format=format)

    if output_path and os.path.exists(output_path):
        try:
            from google.colab import files
            files.download(output_path)

            # Download additional files for OBJ
            if format == 'obj':
                mtl_path = output_path.replace('.obj', '.mtl')
                texture_path = output_path.replace('.obj', '_texture.png')

                if os.path.exists(mtl_path):
                    files.download(mtl_path)
                if os.path.exists(texture_path):
                    files.download(texture_path)

                print("\nBlender Import Instructions for OBJ:")
                print("1. Import the .obj file")
                print("2. Make sure the .mtl and _texture.png files are in the same folder")
                print("3. In Blender, switch to Material Preview or Rendered view")
                print("4. If textures don't appear, manually assign the material in the Material Properties panel")

            elif format == 'ply':
                print("\nBlender Import Instructions for PLY:")
                print("1. Import the .ply file")
                print("2. Select the imported object")
                print("3. Go to Material Properties tab (sphere icon)")
                print("4. Click 'New' to create a new material")
                print("5. In Shader Editor, add these nodes:")
                print("   - Add > Input > Attribute node")
                print("   - Set Attribute node Name to 'Col' (vertex color attribute)")
                print("   - Connect Attribute Color output to Principled BSDF Base Color input")
                print("6. Switch to Material Preview or Rendered viewport shading")
                print("Alternative: Try switching viewport shading to 'Vertex Paint' mode")

            elif format == 'stl':
                print("\nBlender Import Instructions for STL:")
                print("1. Import the .stl file")
                print("2. Note: STL doesn't support colors - you'll need to add materials manually")

            print(f"\nFile ready for download!")

        except ImportError:
            print(f"\nFile exported to: {output_path}")

    return output_path

# Add these methods to your VoxelBasedImage3DConverter class
VoxelBasedImage3DConverter.export_voxel_mesh = export_voxel_mesh
VoxelBasedImage3DConverter.download_mesh = download_mesh
VoxelBasedImage3DConverter.download_mesh_for_blender = download_mesh_for_blender

print("Fixed export_voxel_mesh function loaded!")
print("\nKey fixes:")
print("1. smoothing.filter_laplacian() returns vertices array, not mesh object")
print("2. Added proper error handling for smoothing and simplification")
print("3. Added fallback smoothing methods if Laplacian fails")
print("\nNow run your export command:")

In [ ]:
result = upload_and_process_voxel(
    remove_bg=True,
    visualization_mode='mesh',  # Smooth sphere
    preserve_width=True,
    auto_depth=True  # Will detect sphere and create proper ball
)
if result['success']:
    converter.download_mesh_for_blender(result['voxel_grid'], format='ply')

In [ ]:
result = upload_and_process_voxel(
    remove_bg=True,
    visualization_mode='cubes',  # LEGO blocks
    preserve_width=True,  # Keeps gun proportions
    auto_depth=True  # Will detect "ultra_thin" and minimize depth
)